In [14]:
# zipminator/zipit.py

import pandas as pd
import pyzipper
import time
import threading
import getpass
import os
import hashlib
import random
import string
from typing import List


class Zipndel:
    """Class for compressing and encrypting Pandas DataFrames and deleting the original file.

    Attributes:
        file_name (str): The name of the file to be written, default is 'df'.
        file_format (str): The file format of the file to be written, default is 'csv'.
        self_destruct_time (tuple): A tuple of (hours, minutes, seconds) until self-destruct, default is (672, 0, 0).
        password (str): The password to use for the zip file, default is None.
        encryption_algorithm (str): The encryption algorithm to use for the zip file, default is 'AES'.
        mask_columns (list): The list of columns to mask, default is None.
        anonymize_columns (list): The list of columns to anonymize, default is None.
        compliance_check (bool): Whether to perform a compliance check on the data, default is False.
        audit_trail (bool): Whether to keep an audit trail, default is False.

    Methods:
        mask_columns(df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
            Mask sensitive data in the specified DataFrame columns by applying a SHA-256 hash function.

        anonymize_columns(df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
            Anonymize sensitive data in the specified DataFrame columns by replacing it with random characters.

        zipit(df: pd.DataFrame) -> None:
            Write the input DataFrame to a file, create a zip file with the written file, set a password for the zip file,
            and delete the written file.

        self_destruct(hours: int, minutes: int, seconds: int) -> None:
            Delete the compressed and encrypted file after a specified amount of time.

        decompress_and_read() -> pd.DataFrame:
            Unzip the file, read it using pandas, and delete the unzipped file.

    Example:
        >>> df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})
        >>> Zipndel(file_name='my_file', password='my_password', mask_columns=['B'], anonymize_columns=['C']).zipit(df)
    """
    def __init__(self, file_name='df', file_format='csv', self_destruct_time=(672, 0, 0), password=None, encryption_algorithm='AES', mask_columns=None, anonymize_columns=None, compliance_check=False, audit_trail=False):
        self.file_name = file_name
        self.file_format = file_format
        self.self_destruct_time = self_destruct_time
        self.password = password
        self.encryption_algorithm = encryption_algorithm
        self.mask_columns = mask_columns
        self.anonymize_columns = anonymize_columns
        self.compliance_check = compliance_check
        self.audit_trail = audit_trail
        
        """
        Initialize the Zipndel object.

        Args:
            file_name (str): The name of the file to be written, default is 'df'.
            file_format (str): The file format of the file to be written, default is 'csv'.
            self_destruct_time (tuple): A tuple of (hours, minutes, seconds) until self-destruct, default is (672, 0, 0).
            password (str): The password to use for the zip file, default is None.
            encryption_algorithm (str): The encryption algorithm to use for the zip file, default is 'AES'.
            mask_columns (list): The list of columns to mask, default is None.
            anonymize_columns (list): The list of columns to anonymize, default is None.
            compliance_check (bool): Whether to perform a compliance check on the data, default is False.
            audit_trail (bool): Whether to keep an audit trail, default is False.
        """

    def zipit(self, df: pd.DataFrame) -> None:
        """
        Write the input DataFrame to a file, create a zip file with the written file, set a password for the zip file,
        and delete the written file.

        Args:
            df (pandas.DataFrame): The DataFrame to compress and encrypt.

        Returns:
            None

        Raises:
            None
        """
        if self.mask_columns is not None:
            df = self._mask_columns(df, self.mask_columns)

        if self.anonymize_columns is not None:
            df = self._anonymize_columns(df, self.anonymize_columns)

        write_func = getattr(df, f'to_{self.file_format}')
        write_func(self.file_name, index=False)

        df_zip = f"{self.file_name}.zip"
        with pyzipper.AESZipFile(df_zip, 'w', compression=pyzipper.ZIP_DEFLATED, encryption=getattr(pyzipper, f'WZ_{self.encryption_algorithm}')) as zf:
            if self.password is None:
                self.password = getpass.getpass('Enter password: ')
            zf.setpassword(self.password.encode('utf-8'))
            zf.write(self.file_name)

        os.remove(self.file_name)

        if self.self_destruct_time and self.self_destruct_time != False:
            t = threading.Thread(target=self._self_destruct, args=self.self_destruct_time)
            t.start()

    def _mask_columns(self, df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
        """Mask sensitive data in the specified DataFrame columns by applying a SHA-256 hash function.

        Args:
            df (pandas.DataFrame): The DataFrame to mask sensitive data in.
            columns (list): A list of strings specifying the names of the columns to mask.

        Returns:
            pandas.DataFrame: A copy of the input DataFrame with the specified columns masked.
        """
        df = df.copy()
        for col in columns:
            df[col] = df[col].apply(
                lambda x: hashlib.sha256(str(x).encode()).hexdigest())
        return df

    def _anonymize_columns(self, df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
        """Anonymize sensitive data in the specified DataFrame columns by replacing it with random characters.

        Args:
            df (pandas.DataFrame): The DataFrame to anonymize sensitive data in.
            columns (list): A list of strings specifying the names of the columns to anonymize.

        Returns:
            pandas.DataFrame: A copy of the input DataFrame with the specified columns anonymized.
        """
        df = df.copy()
        for col in columns:
            df[col] = df[col].apply(lambda x: ''.join(
                random.choices(string.ascii_uppercase + string.digits, k=10)))
        return df

    def _self_destruct(self, hours: int, minutes: int, seconds: int) -> None:
        """Delete the compressed and encrypted file after a specified amount of time has elapsed.

        Args:
            hours (int): The number of hours until file deletion.
            minutes (int): The number of minutes until file deletion.
            seconds (int): The number of seconds until file deletion.

        Returns:
        None
        """
        df_zip = f"{self.file_name}.zip"
        self_destruct_time = time.time() + hours * 60 * 60 + minutes * 60 + seconds
        while True:
            if time.time() > self_destruct_time:
                os.remove(df_zip)
                break
            time.sleep(5)

# zipminator/unzipit.py

class Unzipndel:
    
    def __init__(self, file_name='df', file_format='csv'):
        self.file_name = file_name
        self.file_format = file_format

    def unzipit(self):
        """Unzip the file, read it using pandas, and delete the unzipped file.

        Returns:
            pd.DataFrame: A pandas dataframe containing the unzipped and read data.

        Raises:
            RuntimeError: If the password is incorrect or the file cannot be unzipped.
        """
        password = getpass.getpass('Password: ')
        with pyzipper.AESZipFile(f"{self.file_name}.zip") as zf:
            zf.setpassword(password.encode())
            zf.extract(self.file_name)

        read_func = getattr(pd, f'read_{self.file_format}')
        df = read_func(self.file_name)

        os.remove(self.file_name)

        return df

In [16]:
# create a pandas DataFrame with sensitive info
df = pd.DataFrame({
    'ident': [150706, 150707, 150708, 150708, 150709],
    'navn': ['Mo', 'Kari', 'Ola', 'Olga', 'Ali'],
    'email': ['mo@nav.no', 'kari@nav.no', 'ola@nav.no', 'olga@nav.no', 'ali@nav.no'],
    'phone': ['+4798079896', '+4798765434', '+4799887766', '+4745983421', '+4745989933'],
    'addresse': ['Moveien 9', 'Kari gate 34', 'Ola parken 17', 'Olga smuget 55', 'Ali alé 99'],
    'by': ['Oslo', 'Bærum', 'Asker', 'Oslo', 'Oslo'],
    'fylke': ['Oslo', 'Viken', 'Viken', 'Oslo', 'Oslo'],
    'zip': ['0456', '1550', '1750', '0954', '0134'],
    'ansatt_siden': ['2020-01-01', '2019-01-02', '2017-01-03', '2022-01-04', '2022-01-05'],
    'lonn': [780000, 820000, 915000, 825000, 917580]
})

# Initialize a Zipndel object with custom values
zipper = Zipndel(
    file_name='df',
    file_format='csv',
    self_destruct_time=(0, 5, 0),
    encryption_algorithm='AES',
    mask_columns=['email', 'phone'],
    anonymize_columns=['addresse', 'lonn'],
    compliance_check=True,
    audit_trail=True
)

zipper.zipit(df)
df = Unzipndel().unzipit()
df

Enter password:  ········
Password:  ········


,ident,navn,email,phone,addresse,by,fylke,zip,ansatt_siden,lonn
0,150706,Mo,337fbccd2639a442e418a8468f502088f76760685e7937...,91fab6f7870a67a1e0d7df582fa093651e89b928f04298...,67C60FEY08,Oslo,Oslo,456,2020-01-01,4JRJC34KEB
1,150707,Kari,b394a6c9d8bc8757e0b5c5ae8fc4c9367f491ab277b6b6...,81f45b5a8e44c620ae29a1b7cce316e3a1b9421441d6e9...,R3TAKM9U32,Bærum,Viken,1550,2019-01-02,AHX5OMET4P
2,150708,Ola,4ea56fd6aa5e00ac9dbb227ae6340be5124e697a384b75...,1172e3725453f484dcbc0e508d4a46682cf4569cbe75a5...,BINM8SRQ2J,Asker,Viken,1750,2017-01-03,VX7VAVT7AZ
3,150708,Olga,7599e434b49a98898e394b7dc8bf3d4cad961520a5abe4...,4351ade48e331857e87812f6da4ff25ffad291ac3a41b2...,J0XRA5003F,Oslo,Oslo,954,2022-01-04,3PAX1ZP4BK
4,150709,Ali,9844050e343edbbc791c393e11e2ce52dd00740749fb7f...,d95e8ccbe88c159fed81ad29cd480970a856449155c6e8...,2KPP8RHSS6,Oslo,Oslo,134,2022-01-05,HKJ1F0S47X


Exception in thread Thread-10:
Traceback (most recent call last):
  File "/Users/m0/miniconda3/envs/zmenv/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/Users/m0/miniconda3/envs/zmenv/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/93/6sy7vf8142969b_8w873sxcc0000gn/T/ipykernel_79013/3189343210.py", line 159, in _self_destruct
FileNotFoundError: [Errno 2] No such file or directory: 'df.zip'
Exception in thread Thread-11:
Traceback (most recent call last):
  File "/Users/m0/miniconda3/envs/zmenv/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/Users/m0/miniconda3/envs/zmenv/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/93/6sy7vf8142969b_8w873sxcc0000gn/T/ipykernel_79013/3189343210.py", line 159, in _self_destruct
FileNotFoundError: [Errno 2] No such file or directory: 'df.zip'


In [10]:
# create a pandas DataFrame with sensitive info
df = pd.DataFrame({
    'ident': [150706, 150707, 150708, 150708, 150709],
    'navn': ['Mo', 'Kari', 'Ola', 'Olga', 'Ali'],
    'email': ['mo@nav.no', 'kari@nav.no', 'ola@nav.no', 'olga@nav.no', 'ali@nav.no'],
    'phone': ['+4798079896', '+4798765434', '+4799887766', '+4745983421', '+4745989933'],
    'addresse': ['Moveien 9', 'Kari gate 34', 'Ola parken 17', 'Olga smuget 55', 'Ali alé 99'],
    'by': ['Oslo', 'Bærum', 'Asker', 'Oslo', 'Oslo'],
    'fylke': ['Oslo', 'Viken', 'Viken', 'Oslo', 'Oslo'],
    'zip': ['0456', '1550', '1750', '0954', '0134'],
    'ansatt_siden': ['2020-01-01', '2019-01-02', '2017-01-03', '2022-01-04', '2022-01-05'],
    'lonn': [780000, 820000, 915000, 825000, 917580]
})

# Initialize a Zipndel object with custom values
zipper = Zipndel(
    file_name='df',
    file_format='csv',
    self_destruct_time=(0, 5, 0),
    password='NAV123',
    encryption_algorithm='AES',
    mask_columns=['email', 'phone'],
    anonymize_columns=['addresse', 'lonn'],
    compliance_check=True,
    audit_trail=True
)

# Compress and encrypt the DataFrame
zipper.zipit(df)




In [11]:
# Initialize an Unzipndel object
unzipper = Unzipndel(file_name='df', file_format='csv')

# Decompress and read the data from the encrypted file
df_unzipped = unzipper.unzipit()

# Display the unzipped DataFrame
print(df_unzipped)

Password:  ········


    ident  navn                                              email  \
0  150706    Mo  337fbccd2639a442e418a8468f502088f76760685e7937...   
1  150707  Kari  b394a6c9d8bc8757e0b5c5ae8fc4c9367f491ab277b6b6...   
2  150708   Ola  4ea56fd6aa5e00ac9dbb227ae6340be5124e697a384b75...   
3  150708  Olga  7599e434b49a98898e394b7dc8bf3d4cad961520a5abe4...   
4  150709   Ali  9844050e343edbbc791c393e11e2ce52dd00740749fb7f...   

                                               phone    addresse     by  \
0  91fab6f7870a67a1e0d7df582fa093651e89b928f04298...  J7SW1I8MLK   Oslo   
1  81f45b5a8e44c620ae29a1b7cce316e3a1b9421441d6e9...  CVI8JJ5TWM  Bærum   
2  1172e3725453f484dcbc0e508d4a46682cf4569cbe75a5...  09UBJRUXIK  Asker   
3  4351ade48e331857e87812f6da4ff25ffad291ac3a41b2...  3UJO0NH9H2   Oslo   
4  d95e8ccbe88c159fed81ad29cd480970a856449155c6e8...  MD8RW9SAAT   Oslo   

   fylke   zip ansatt_siden        lonn  
0   Oslo   456   2020-01-01  CEEXDFUADH  
1  Viken  1550   2019-01-02  ANNAD4OWMO  
2 

Exception in thread Thread-7:
Traceback (most recent call last):
  File "/Users/m0/miniconda3/envs/zmenv/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/Users/m0/miniconda3/envs/zmenv/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/93/6sy7vf8142969b_8w873sxcc0000gn/T/ipykernel_79013/2327203709.py", line 159, in _self_destruct
FileNotFoundError: [Errno 2] No such file or directory: 'df.zip'
Exception in thread Thread-8:
Traceback (most recent call last):
  File "/Users/m0/miniconda3/envs/zmenv/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/Users/m0/miniconda3/envs/zmenv/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/93/6sy7vf8142969b_8w873sxcc0000gn/T/ipykernel_79013/2327203709.py", line 159, in _self_destruct
FileNotFoundError: [Errno 2] No such file or directory: 'df.zip'
